In [1]:
import math

BITS = 8
BITS_SIZE = 2 ** BITS
BITS_MASK = BITS_SIZE - 1
MAX_PERMS = math.factorial(BITS)

RSA_PQ_BITS = 8
RSA_PQ_SIZE= 2 ** RSA_PQ_BITS
RSA_PQ_BYTES = RSA_PQ_BITS // 8

RSA_N_BITS = RSA_PQ_BITS * 2
RSA_N_SIZE = 2 ** RSA_N_BITS
RSA_N_BYTES = RSA_N_BITS // 8

def bit_sub_encrypt(input: int, key: int) -> int:
    key %= MAX_PERMS
    output = 0
    used_bits = 0
    for old_bit_pos in range(BITS):
        choice_count = BITS - old_bit_pos
        zeroes_to_bit_pos = key // math.factorial(choice_count - 1) % choice_count
        cursor = 0
        while (bit_value := used_bits >> cursor & 1) == 1 or zeroes_to_bit_pos > 0:
            cursor += 1
            zeroes_to_bit_pos -= ~bit_value & 1
        output |= (input >> old_bit_pos & 1) << cursor
        used_bits |= 1 << cursor
    return output

def bit_sub_decrypt(input: int, key: int) -> int:
    key %= MAX_PERMS
    output = 0
    used_bits = 0
    for old_bit_pos in range(BITS):
        choice_count = BITS - old_bit_pos
        zeroes_to_bit_pos = key // math.factorial(choice_count - 1) % choice_count
        cursor = 0
        while (bit_value := used_bits >> cursor & 1) == 1 or zeroes_to_bit_pos > 0:
            cursor += 1
            zeroes_to_bit_pos -= ~bit_value & 1
        output |= (input >> cursor & 1) << old_bit_pos
        used_bits |= 1 << cursor
    return output

def bit_shift(input: int, sh: int) -> int:
    return (input >> sh % BITS | input << BITS - sh % BITS) & (2 ** BITS - 1)

def vigenere_encrypt(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return (input + key[i % len(key)]) % BITS_SIZE

def vigenere_decrypt(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return (input - key[i % len(key)]) % BITS_SIZE

def vernam_cipher(input: int, key: bytes, i: int) -> int:
    if len(key) == 0:
        return input
    return input ^ key[i % len(key)]

def rsa_encrypt(input: int, n: int, e: int) -> int:
    return pow(input, e, n)

def rsa_decrypt(input: int, n: int, d: int) -> int:
    return pow(input, d, n)

In [2]:
import math
import ipywidgets as widgets
import random

# RSA ALGORITHM
def rsa_key_generator() -> (int, int, int):
    # Fermat primality test
    def is_prime(n):
        if n == 2 or n == 3:
            return True
        if n % 2 == 0:
            return False
        for _ in range(100):
            a = random.randint(2, n - 2)
            if pow(a, n - 1, n) != 1:
                return False
        return True

    # Generate a prime number for RSA
    def generate_prime(n):
        while True:
            p = random.randint(2**(n-1), 2**n)
            if is_prime(p):
                return p

    def generate_e(phi_n):
        while True:
            e = random.randint(2, phi_n - 1)
            if math.gcd(e, phi_n) == 1:
                return e

    def generate_d(e, phi_n):
        def extended_gcd(a, b):
            if b == 0:
                return a, 1, 0
            gcd, x, y = extended_gcd(b, a % b)
            return gcd, y, x - (a // b) * y

        _, d, _ = extended_gcd(e, phi_n)
        d %= phi_n
        return d
    
    # Generate p and q
    p = generate_prime(RSA_PQ_BITS)
    q = None
    while True:
        q = generate_prime(RSA_PQ_BITS)
        if q != p:
            break
    # Calculate n
    n = p * q
    # Calculate phi(n)
    phi_n = (p - 1) * (q - 1)
    e = generate_e(phi_n)
    d = generate_d(e, phi_n)
    return (n, e, d)


# rsa_n, e, d = rsa_key_generator()

# input = 121
# encrypted = pow(input, e, rsa_n)
# decrypted = pow(encrypted, d, rsa_n)
# print(input, encrypted, decrypted)
# print(rsa_n, e, d)


In [3]:
import random
import base64
from dataclasses import dataclass

@dataclass
class CipherKey:
    bit_cipher_key: int # 0 - 8!-1
    shift_init: int # 0-7
    shift_rate: int # 0-7
    vigenere_key: bytes # 0-255[<255]
    vernam_key: bytes # 0-255[<255]
    rsa_n: int # 16 bits
    rsa_key: int # 8 bits

    def __post_init__(self):
        if not (0 <= self.bit_cipher_key < MAX_PERMS):
            raise ValueError("bit_cipher_key is out of range")
        if not (0 <= self.shift_init < BITS):
            raise ValueError("shift_init is out of range")
        if not (0 <= self.shift_rate < BITS):
            raise ValueError("shift_rate is out of range")
        if len(self.vigenere_key) >= BITS_SIZE:
            raise ValueError("vigenere_key is out of range")
        if len(self.vernam_key) >= BITS_SIZE:
            raise ValueError("vernam_key is out of range")
        if not (0 <= self.rsa_n < RSA_N_SIZE):
            raise ValueError("rsa n is out of range")
        if not (0 <= self.rsa_key < RSA_N_SIZE):
            raise ValueError("rsa key is out of range")
    
    def to_bytes(self) -> bytes:
        return self.bit_cipher_key.to_bytes(2, byteorder='big') \
            + bytes([self.shift_init, self.shift_rate, len(self.vigenere_key)]) \
            + self.vigenere_key \
            + bytes([len(self.vernam_key)]) \
            + self.vernam_key \
            + self.rsa_n.to_bytes(RSA_N_BYTES, byteorder='big') \
            + self.rsa_key.to_bytes(RSA_N_BYTES, byteorder='big')

    def to_base64(self) -> str:
        return base64.b64encode(self.to_bytes()).decode('utf-8')

    @classmethod
    def from_random(cls):
        bit_cipher_key = random.randint(0, MAX_PERMS - 1)
        shift_init = random.randint(0, BITS - 1)
        shift_rate = random.randint(0, BITS - 1)
        vigenere_key = bytes([random.randint(0, 255) for _ in range(random.randint(0, 255))])
        vernam_key = bytes([random.randint(0, 255) for _ in range(random.randint(0, 255))])
        n, e, d = rsa_key_generator()
        return (
            cls(bit_cipher_key, shift_init, shift_rate, vigenere_key, vernam_key, n, e),
            cls(bit_cipher_key, shift_init, shift_rate, vigenere_key, vernam_key, n, d),
        )

    @classmethod
    def from_bytes(cls, b: bytes):
        def get_bytes(b_iter, size):
            return bytes([next(b_iter) for _ in range(size)])
        
        b_iter = iter(b)        

        return cls(
            bit_cipher_key = int.from_bytes(get_bytes(b_iter, 2), byteorder='big'),
            shift_init = next(b_iter) % BITS,
            shift_rate = next(b_iter) % BITS,
            vigenere_key = get_bytes(b_iter, next(b_iter)),
            vernam_key = get_bytes(b_iter, next(b_iter)),
            rsa_n = int.from_bytes(get_bytes(b_iter, RSA_N_BYTES), byteorder='big'),
            rsa_key = int.from_bytes(get_bytes(b_iter, RSA_N_BYTES), byteorder='big'),
        )

    @classmethod
    def from_base64(cls, b64: str):
        return cls.from_bytes(base64.b64decode(b64))


class Cipher:
    def __init__(self, key: CipherKey = None) -> None:
        self.i = 0
        self.key = key
    
    def set_key(self, key: CipherKey) -> None:
        self.key = key
    
    def reset(self) -> None:
        self.i = 0
    
    def encrypt(self, x: int) -> bytes:
        if self.key is None:
            raise Exception('Key is Omitted')
        x = bit_shift(x, self.key.shift_init + self.i * self.key.shift_rate)
        x = bit_sub_encrypt(x, self.key.bit_cipher_key)
        x = vigenere_encrypt(x, self.key.vigenere_key, self.i)
        x = vernam_cipher(x, self.key.vernam_key, self.i)
        y = rsa_encrypt(x, self.key.rsa_n, self.key.rsa_key).to_bytes(RSA_N_BYTES, byteorder='big')
        self.i += 1
        return y

    def decrypt(self, y: bytes) -> int:
        if self.key is None:
            raise Exception('Key is Omitted')
        x = rsa_decrypt(int.from_bytes(y, byteorder='big'), self.key.rsa_n, self.key.rsa_key)
        x = vernam_cipher(x, self.key.vernam_key, self.i)
        x = vigenere_decrypt(x, self.key.vigenere_key, self.i)
        x = bit_sub_decrypt(x, self.key.bit_cipher_key)
        x = bit_shift(x, -self.key.shift_init - self.i * self.key.shift_rate)
        self.i += 1
        return x
# print(CipherKey(1, 2, 3, 'hello world'.encode(), 'hello thing'.encode()))

In [4]:
import hashlib

input_filename = './Output/Test.txt'
encrypted_filename = './Output/Encrypted.txt'
decrypted_filename = './Output/Decrypted.txt'

key_encrypt, key_decrypt = CipherKey.from_random()

with open(input_filename, 'rb') as input_file:
    with open(encrypted_filename, 'wb') as output_file:
        cipher = Cipher(key_encrypt)
        # copy by byte to encrypted_filename
        while (byte := input_file.read(1)):
            output_file.write(cipher.encrypt(int.from_bytes(byte, byteorder='big')))

with open(encrypted_filename, 'rb') as input_file:
    with open(decrypted_filename, 'wb') as output_file:
        cipher = Cipher(key_decrypt)
        # copy by byte to decrypted_filename
        while (ecrypted_bytes := input_file.read(RSA_N_BYTES)):
            output_file.write(cipher.decrypt(ecrypted_bytes).to_bytes(1, byteorder='big'))


def get_file_hash(filename):
    with open(filename, 'rb') as file:
        data = file.read()
        hash_object = hashlib.sha256(data)
        return hash_object.hexdigest()

input_hash = get_file_hash(input_filename)
encrypted_hash = get_file_hash(encrypted_filename)
decrypted_hash = get_file_hash(decrypted_filename)

print("Key Encrypt:", key_encrypt.to_base64())
print("Key Decrypt:", key_decrypt.to_base64())
print("Input File Hash:", input_hash)
print("Encrypted File Hash:", encrypted_hash)
print("Decrypted File Hash:", decrypted_hash)


Key Encrypt: TW0HAoS70xLa2r8l+OiueYhL9/Uj6qqkfULFbVTQRYqUY+MxMJPu0ua+rCMi1wWsAABeX4fpqQQjG3UXSdcxl3fp9L7eEpDPlhZaGJpDM5B0aQnccpa0yjLwKac0H9qcMmgm4KRFvxw9pudEv3Kn+tdgVcciKyGOmFniEL7mOrUay0QYBZDKofrm/f1krTNldU+6l/LTq9g4M7PJVI15xnVlhS4dK0XqWcU+KDWJmvr9mSeoJS27FEBbvxiOaqSUiZV7WsmMH2xxulX2Oee6sp4PtvNuV9jWTR3y02b1kVmVyYcCG6CR0IG7UBBM22UFI9d8xyV7FVVukUHjJeGvo+bN/cOgTJj0FVfei+BgkYvRp2FEVlWYPckI84oxKqdvEVKkgJ9QUC0U0I9Jz5/FoQ0bUte8B9cLgPkvpxstS1Y8iZ/qFrrgCkAm1JqHTH4Px2vtih4qIDqZ2jk4tBkA3SHtnZx22vX/CH4pgooVAD56UXAl
Key Decrypt: TW0HAoS70xLa2r8l+OiueYhL9/Uj6qqkfULFbVTQRYqUY+MxMJPu0ua+rCMi1wWsAABeX4fpqQQjG3UXSdcxl3fp9L7eEpDPlhZaGJpDM5B0aQnccpa0yjLwKac0H9qcMmgm4KRFvxw9pudEv3Kn+tdgVcciKyGOmFniEL7mOrUay0QYBZDKofrm/f1krTNldU+6l/LTq9g4M7PJVI15xnVlhS4dK0XqWcU+KDWJmvr9mSeoJS27FEBbvxiOaqSUiZV7WsmMH2xxulX2Oee6sp4PtvNuV9jWTR3y02b1kVmVyYcCG6CR0IG7UBBM22UFI9d8xyV7FVVukUHjJeGvo+bN/cOgTJj0FVfei+BgkYvRp2FEVlWYPckI84oxKqdvEVKkgJ9QUC0U0I9Jz5/FoQ0bUte8B9cLgPkvpxstS1Y8iZ/qFrrgCkAm1JqHTH4Px2vtih4qIDqZ2jk4tBkA3SHtnZx22